# Advent of Code 2023

https://adventofcode.com/2023

## Day 1

In [1]:
sample = '1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet'

"1abc2\npqr3stu8vwx\na1b2c3d4e5f\ntreb7uchet"

In [41]:
def coordinates(haystack)
  digits = haystack.scan(/\d/).to_a
  [digits[0], digits[-1]].join.to_i
end

:coordinates

In [40]:
def checksum(data)
  data.split("\n").map { |line| coordinates(line) }.sum
end

:checksum

In [100]:
puzzle_1 = File.read('2023-d01a.txt')
checksum(puzzle_1)

55621

In [79]:
sample_2 = 'two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen'

# NUMBERS = {
#   'one': '1',
#   'two': '2',
#   'three': '3',
#   'four': '4',
#   'five': '5',
#   'six': '6',
#   'seven': '7',
#   'eight': '8',
#   'nine': '9'
# }

# def translate(data)
#   NUMBERS.each do |name, numeral|
#     data.gsub!(name.to_s, numeral)
#   end
#   data
# end

# translate(sample_2).split("\n").map { |line| coordinates(line) }

"two1nine\neightwothree\nabcone2threexyz\nxtwone3four\n4nineeightseven2\nzoneight234\n7pqrstsixteen"

In [101]:
puzzle_2 = File.read('2023-d01a.txt')


"mxmkjvgsdzfhseightonetwoeight7\n3five4s84four9rtbzllggz\n75sevenzdrpkv1onetwo\n3q7ctkghhqkpb5four\nccxpblrgrjxcgrhjxfmtwonine8eightzzrdn4\n6lqjrhbnxxcqlpnmjsthreesixxsxcgqsxmdx7\n35llbrhh\nfivethreejcqpfqmcmvjlmhlbzpxlsmktzkmtmv8p\n3369eightnine89\nonesix4qqnlzdsevennmmrcgkndlsgm3\ntwo4xgdjdqtcjk1threelkjdxvfivesix\nsixsix6njhqrnine\n4skbhsbtqc\none11fpkjsix\nkgtkgdjtwo6mmklqc\nkqfqq39gnncltrtpgtwo5bxmx\njdqgmr5jtz\n6hltxnninesmfzpnqjnkcrstv\ntwotwo6eightqcncghjvqfcv5\n4ppzhcvqrnkqonehlhlcqmrq\n1xldgvsix67\n2six1bsxxgbonefour\nstwone3qlrfive314sixsix\ntfkbqvreight34r\nfive4eight\nrvsmznr4eightjqckcr6nine6rhpnmxrhdc\nxpqldjttwompcfseven8seven8two\nfourclkthghllzlhrs31\n2hpmvfbm8seven5v58\n3qnfqtgndveightthree\nfivethreervmhvzgeightvzc63\nhlpsp9nqnsvhdcfourfivestcz\nnineninedtfivefive4\ngb9twoseven\n1fourfive4seveng\ntmlsix2fiveninefourgjltplkfcnine\none3sdrckqtzdzgvtl7vtwo\ns2fdklslhzrxktzzttxdtnx1\nthree8vhssix9hhtrxkbpjp39mcchnpp\n37three5btqxsqkszchfivebvbbddssvc\nz5onefour\n3eightr

In [128]:
puzzle_2 = File.read('2023-d01a.txt')

sample_2 = 'two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen'

NUMBERS = {
  'one' => '1',
  'two' => '2',
  'three' => '3',
  'four' => '4',
  'five' => '5',
  'six' => '6',
  'seven' => '7',
  'eight' => '8',
  'nine' => '9'
}

NUMBER_MATCHER = ['\d', *NUMBERS.keys].join('|')
SCANNABLE_NUMBER_MATCHER = Regexp.new("(?=(#{NUMBER_MATCHER}))")

def verbal_checksum(data)
  data.split("\n").map do |line|
    first_digit_match, *_other_matches, last_digit_match = line.scan(SCANNABLE_NUMBER_MATCHER).flatten
    first_digit = NUMBERS[first_digit_match] || first_digit_match
    last_digit = NUMBERS[last_digit_match] || last_digit_match || first_digit
    [first_digit, last_digit].join.to_i
  end
end

# verbal_checksum(sample_2)

verbal_checksum(puzzle_2)

(irb):10: warning: already initialized constant Object::NUMBERS
(irb):10: warning: previous definition of NUMBERS was here
(irb):22: warning: already initialized constant Object::NUMBER_MATCHER
(irb):22: warning: previous definition of NUMBER_MATCHER was here
(irb):23: warning: already initialized constant Object::SCANNABLE_NUMBER_MATCHER
(irb):23: warning: previous definition of SCANNABLE_NUMBER_MATCHER was here


[87, 39, 72, 34, 24, 67, 35, 58, 39, 13, 26, 69, 44, 16, 26, 35, 55, 69, 25, 41, 17, 24, 26, 84, 58, 46, 22, 41, 28, 33, 53, 95, 94, 97, 17, 69, 12, 21, 39, 35, 54, 32, 99, 44, 59, 79, 92, 77, 34, 39, 88, 33, 72, 88, 76, 94, 44, 15, 11, 63, 43, 15, 79, 17, 39, 88, 81, 59, 23, 46, 11, 54, 73, 25, 65, 45, 42, 39, 88, 27, 64, 74, 39, 82, 65, 34, 81, 24, 19, 98, 18, 55, 72, 26, 31, 79, 84, 32, 66, 38, 41, 42, 63, 96, 88, 12, 59, 62, 12, 46, 44, 36, 84, 74, 31, 54, 29, 44, 23, 84, 15, 89, 47, 66, 16, 85, 12, 54, 28, 95, 39, 78, 56, 37, 97, 31, 34, 72, 23, 13, 62, 89, 67, 93, 17, 45, 55, 27, 76, 23, 89, 46, 95, 36, 81, 27, 12, 61, 51, 14, 22, 22, 61, 88, 41, 41, 66, 12, 16, 59, 32, 35, 12, 16, 51, 57, 94, 44, 72, 83, 99, 65, 84, 63, 41, 38, 98, 85, 19, 84, 59, 88, 38, 22, 53, 85, 46, 46, 54, 13, 26, 69, 62, 99, 67, 98, 59, 19, 33, 24, 37, 81, 67, 72, 11, 86, 93, 11, 42, 31, 35, 11, 73, 41, 74, 41, 78, 57, 41, 88, 84, 46, 14, 73, 49, 48, 33, 39, 81, 14, 62, 46, 89, 17, 69, 32, 24, 44, 99, 13,

In [113]:
verbal_checksum(puzzle_2)

53592

In [126]:
# scan starts looking for the next match after the end of the previous match.
REGULAR_MATCHER = /(\D\D\D)/
'abcde8fgh'.scan(REGULAR_MATCHER)
# [["abc"], ["fgh"]]

# lookahead matches don't consume the characters they match, so they can overlap.
# scan moves forward by one character after each match.
OVERLAPPING_MATCHER = /(?=(\D\D\D))/
'abcde8fgh'.scan(OVERLAPPING_MATCHER)
# [["abc"], ["bcd"], ["cde"], ["fgh"]]


(irb): warning: already initialized constant Object::OVERLAPPING_MATCHER


[["abc"], ["fgh"]]

## Day 2

In [158]:
d2_sample = 'Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green'

def parse(data)
  data.split("\n").map do |game|
    minimums = {}
    number, results = game.split(':')
    game_id = number.match(/\d+/)[0].to_i

    results.split(';').map do |round|
      counts = {}
      round.split(',').map do |hand|
        number, color = hand.strip.split(' ')
        counts[color] = number.to_i
      end
      minimums.merge!(counts) { |_, old, new| [old, new].max }
    end
    [game_id, minimums]
  end
end

def solve(data)
  parse(data).map do |id, game|
    next id if game['red'] <= 12 && game['green'] <= 13 && game['blue'] <= 14
    0
  end
end

# solve(d2_sample)

d2_input = File.read('2023-d02.txt')
solve(d2_input).sum

2685

In [166]:
def solve_2(data)
  parse(data).sum(0) do |_id, game|
    game.values.reduce(&:*)
  end
end

# solve_2(d2_sample)
solve_2(d2_input)

83707

## Day 3